## Extract Info from Youtube Videos
### Multithreading

## Using urllib2
use this for westeros

In [1]:
import urllib2
import json
from bs4 import BeautifulSoup as soup
import bs4
from time import sleep

In [2]:
my_url = 'https://www.youtube.com/watch?v=e7t4pTNZshA'
page_html= urllib2.urlopen(my_url).read()
soup_out = bs4.BeautifulSoup(page_html)   

In [3]:
# soup_out
# page_html

In [4]:
# page_html

In [5]:
# page= "".join(page_html.split())
# page

In [6]:
def extract_meta(soup_base,prop_name,attribute_name):
    temp = soup_base.find_all('meta',{prop_name:attribute_name})
    for element in temp:
        out =(element.attrs["content"])    
    return out
        
def extract_video_info(my_url=None,result_flag=True):
    #     print(my_url)
    page_html= urllib2.urlopen(my_url).read()
    sleep(0.002)
    #     print(page_html)
    soup_out = bs4.BeautifulSoup(page_html)    
    out_title=extract_meta(soup_out,"name","title").encode('utf-8')
    out_keys=extract_meta(soup_out,"name","keywords").encode('utf-8')

    temp = soup_out.find_all('script',{'type':'application/ld+json'})[0]
    description=extract_meta(soup_out,"itemprop","description").encode('utf-8')
    category=extract_meta(soup_out,"itemprop","genre").encode('utf-8')
    viewcount=extract_meta(soup_out,"itemprop","interactionCount").encode('utf-8')
    twitter_description=extract_meta(soup_out,"name","twitter:description").encode('utf-8')
    channelID=extract_meta(soup_out,"itemprop","channelId").encode('utf-8')
    duration=extract_meta(soup_out,"itemprop","duration").encode('utf-8')
    videoID=extract_meta(soup_out,"itemprop","videoId").encode('utf-8')

    for string in temp.strings:
        a=json.loads(string)
        author = a['itemListElement'][0]['item']['name'].encode('utf-8')    
    if result_flag:
        result = {"title": out_title, "keywords": out_keys,  \
                 "author": author, "category": category,     \
                 "viewcount": viewcount, "description": description, \
                 "twitter_desc": twitter_description,         \
                 "channelID":channelID, "duration":duration,  \
                 "videoID":videoID}
        
        return result

    else:
        
        return out_title, out_keys,author, \
            category,viewcount,description, \
            twitter_description,channelID,duration,videoID
            


In [7]:
try:
    t,k,a,c,v,d,td,chid,duration,videoID = extract_video_info('https://www.youtube.com/watch?v=e7t4pTNZshA',result_flag=False)
    print(t,k,a,c,v,d,td,chid,duration,videoID)
except:
    pass
else:
    pass

('Hattie McDaniel winning Best Supporting Actress', 'Oscars, Academy, Awards, Hattie, McDaniel, Best, Supporting, Actress, Gone, with, the, Wind', 'Oscars', 'Entertainment', '374332', 'Fay Bainter presenting Hattie McDaniel with the Oscar\xc2\xae for Best Supporting Actress for her performance in "Gone With The Wind" at the 12th Academy Awards\xc2\xae in...', 'Fay Bainter presenting Hattie McDaniel with the Oscar\xc2\xae for Best Supporting Actress for her performance in "Gone With The Wind" at the 12th Academy Awards\xc2\xae in...', 'UCb-vZWBeWA5Q2818JmmJiqQ', 'PT1M41S', 'e7t4pTNZshA')


### 1 read csv list of url

In [8]:
import pandas as pd
input_data = pd.read_csv('./sample_vids.txt',sep='\t')

### 2 call extract_video_info to loop on list


In [9]:
i=0
input_data
for text in input_data["VIDID"]:
    test_url='https://www.youtube.com/watch?v=%s' %(text)
    try:
        
        t,k,a,c,v,d,td,chid,duration,videoID = extract_video_info(test_url,result_flag=False)
        print(t,k,a,c,v,d,td,chid,duration,videoID)
    except:
        pass
    else:
        pass


('DIDITH ROMERO in SA DILA NG APOY', 'TAGALOG, MOVIE', 'pilipinomovies', 'Film & Animation', '220546', 'SA DILA NG APOY,1987 * BOLD AND PENE MOVIE* CHECK MY BLOG--http://ratedxxxmovies.blogspot.com', 'SA DILA NG APOY,1987 * BOLD AND PENE MOVIE* CHECK MY BLOG--http://ratedxxxmovies.blogspot.com', 'UCh-eblga9rZwmUTuft22rtw', 'PT3M32S', '0XV3JQBX9sM')
('M\xc3\xa1s Sabe el Diablo | Jencarlos Canela y Gaby Espino hacen el amor | Telemundo', 'Jencarlos Canela, Gaby Espino, TVTelemundo, TelemundoChannel', 'Telemundo FANS', 'Entertainment', '2791458', 'Jencarlos Canela y Gaby Espino en una de las escenas m\xc3\xa1s calientes de todos los tiempos en la televisi\xc3\xb3n.', 'Jencarlos Canela y Gaby Espino en una de las escenas m\xc3\xa1s calientes de todos los tiempos en la televisi\xc3\xb3n.', 'UCcgrOOB6iaytX1nG0UFmgTw', 'PT1M34S', 'EDLwppTAdW8')
('Realistic Earthquake Sound Effect [High Quality]', 'sound, effect, hd, 320kbs, 320, recording, wav, mp3, fx, ambience, ambiance, background, noise, 

### 3 Write to csv

In [10]:
import csv
from time import sleep


In [11]:
# input_data.VIDID.unique
# input_data['VIDID'].drop_duplicates(inplace=True)
input_data=pd.read_csv('/data/raw_data/youtube_video_id/video_id_july.csv')
temp_vidid = input_data['vidid']
print(input_data.vidid.count())

temp_vidid.drop_duplicates(inplace=True)
print(input_data.vidid.count())


942761
379611


## Run the extractor for the csv of videos

- Just replace the **input_csv** variable with the input csv
- Replace **output_csv with the** output csv file
- Modify **input_delimeter** to suit the input csv file
- Modify **output_delimeter** if you want other delimeters
- Replace **input_column** with the column name of the video ID
- Set **row_start** to start the processing from this Row

In [12]:
#!/usr/bin/env python

from multiprocessing import Process, Pool
import time
import urllib2

def millis():
    return int(round(time.time() * 1000))

def http_get(url):
    start_time = millis()
    result = {"url": url, "data": urllib2.urlopen(url, timeout=5).read()}
    print url + " took " + str(millis() - start_time) + " ms"
    return result
  
urls = my_url
pool = Pool(processes=8)


In [13]:
import logging
from urllib2 import urlopen
from threading import Thread
from json import JSONDecoder
 
# Define a crawl function that retrieves data from a url and places the result in results[index]
# The 'results' list will hold our retrieved data
# The 'urls' list contains all of the urls that are to be checked for data

def crawl(url, result, index):
    # Keep everything in try/catch loop so we handle errors
    try:
        data = extract_video_info(url)
        logging.info("Requested..." + url)
        result[index] = data
#         result[index]['row_id'] = index
    except:
        logging.error('Error with URL check!')
        result[index] = {}
    return True

In [14]:
input_column='vidid'
input_data = pd.read_csv('/data/raw_data/youtube_video_id/video_id_july.csv',sep=',')

In [18]:
input_data_july=pd.read_csv('/data/raw_data/youtube_video_id/video_id_july.csv')
input_data_june=pd.read_csv('/data/raw_data/youtube_video_id/video_id_june.csv')
input_data_may=pd.read_csv('/data/raw_data/youtube_video_id/video_id_may.csv')
input_data_april=pd.read_csv('/data/raw_data/youtube_video_id/video_id_april.csv')


In [19]:
input_data = input_data_july[['vidid']]
input_data.count()

vidid    942761
dtype: int64

In [21]:
input_data = input_data.append(input_data_june[['vidid']])
input_data.count()

vidid    1932719
dtype: int64

In [22]:
input_data = input_data.append(input_data_may[['vidid']])
input_data.count()

vidid    3149011
dtype: int64

In [23]:
input_data = input_data.append(input_data_april[['vidid']])
input_data.count()

vidid    4399035
dtype: int64

In [25]:
temp_vidid = input_data[input_column]
temp_vidid.drop_duplicates(inplace=True)
row_start=0 #change this
proc_batch=100
total_rows = len(temp_vidid) #end row, change this

print(total_rows)

# temp_res = extract_video_info(my_url[0])
temp_vidid.head()

1280270


0    HO-R6LGuCR4
1          mOrEe
2    2R06ch5QYYA
3    UsXFWqbvJZw
4    Hf_EtwmA3E8
Name: vidid, dtype: object

In [ ]:
fieldnames = ['title','keywords', 'author','category',\
              'viewcount','description','twitter_desc'\
              ,'channelID','duration','videoID']

with open('temp_0.csv', 'wb') as csvfile:
    writer = csv.DictWriter(csvfile,delimiter='\t',quotechar='"',quoting=csv.QUOTE_NONE, fieldnames=fieldnames)
    writer.writeheader()
    
for i in range(row_start,total_rows,proc_batch):
    if(i>=row_start):
        start_time = millis()

        my_url = []
        for i_sub in range(proc_batch):
            temp = 'https://www.youtube.com/watch?v=%s' %(temp_vidid.iloc[i+i_sub])
            my_url.append(temp)
        results = [{} for x in my_url]

        threads = []
        # In this case 'urls' is a list of urls to be crawled.
        for ii in range(len(my_url)):
            # We start one thread per url present.
            process = Thread(target=crawl, args=[my_url[ii], results, ii])
            process.start()
            threads.append(process)

        # We now pause execution on the main thread by 'joining' all of our started threads.
        # This ensures that each has finished processing the urls.
        for process in threads:
            process.join()

        with open('temp_0.csv', 'ab') as csvfile:
            writer = csv.DictWriter(csvfile,delimiter='\t', fieldnames=fieldnames)
            for row in results:
                writer.writerow(row)    
        print(i,str(millis() - start_time), 'msec')


(0, '171527', 'msec')
(64, '121803', 'msec')
(128, '120660', 'msec')
